In [ ]:
import pandas as pd
import os
import datetime

file_master_dir = './data-master/'
file_master_list = [f for f in os.listdir(file_master_dir) if os.path.isfile(os.path.join(file_master_dir, f))]
file_master_list

file_under_dir = './data-under/'
file_under_list = [f for f in os.listdir(file_under_dir) if os.path.isfile(os.path.join(file_under_dir, f))]
file_under_list

In [ ]:
def get_day_of_week(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d').strftime('%A')

In [ ]:
def preprocess(file_path, target):
    import re
    original_data = pd.read_excel(file_path, header=None)

    # 새로운 데이터프레임을 만들고, A11부터 A34까지의 값을 'time' 헤더로 넣어줍니다
    new_data = pd.DataFrame()
    new_data["time"] = original_data.iloc[10:34, 0]

    # 나머지 헤더와 범위에 대한 정보를 정리합니다
    columns_info = {}
    
    if target=='master':
        columns_info = {
            "SV-2": (10, 9),
            "SV-5": (10, 20),
            "SV-6": (10, 31),
            "HV-M1": (10, 38),
            "HV-M2": (10, 43),
            "HV-NM": (10, 48),
            "HV-EM": (10, 53),
            "중앙P/P": (10, 58),
            "전기전자컴퓨터공학동": (10, 63),
            "신소재공학": (10, 68),
            "생명과학동": (10, 73),
            "기계공학동": (10, 78),
            "LG도서관": (10, 83),
            "창업B동": (10, 88),
            "금호관": (10, 93),
            "냉각수순환펌프1,2,3": (10, 98),
            "냉각수순환펌프4,5,6": (10, 103),
            "기혼자아파트E동": (10, 108),
            "기숙사9동": (10, 113),
            "고등광/극초단": (10, 118),
            "신소재공학동(E)": (10, 123),
            "전기전자컴퓨터공학동(E)": (10, 128),
            "생명과학동(E)": (10, 133),
            "기계공학동(E)": (10, 138),
            "LG도서관(E)": (10, 143),
            "중앙P/P(E)": (10, 148),
            "고등광연구소(E)": (10, 153),
        }
    elif target=='under':
        columns_info = {
            "SV-2": (10, 9),
            "SV-5": (10, 20),
            "SV-6": (10, 31),
            "SV-7": (10, 42),
            "HV-NM1": (10, 49),
            "HV-NM2": (10, 54),
            "고압콘덴샤": (10, 59),
            "신재생에너지동": (10, 64),
            "대학B동": (10, 69),
            "대학기숙사A동": (10, 74),
            "제2학생회관": (10, 79),
            "학사P/P": (10, 84),
            "교원아파트": (10, 89),
            "대학C동": (10, 94),
            "중앙연구기기센터": (10, 99),
            "대학A동": (10, 104),
            "다산빌딩": (10, 109),
            "산학협력연구동": (10, 114),
            "신재생에너지동(E)": (10, 119),
            "대학B동(E)": (10, 124),
            "대학기숙사A동(E)": (10, 129),
            "학사P/P(E)": (10, 134),
            "제2학생회관(E)": (10, 139),
            "교원아파트(E)": (10, 144),
            "대학C동(E)": (10, 149),
            "중앙연구기기센터(E)": (10, 154),
            "대학A동(E)": (10, 159),
            "다산빌딩(E)": (10, 164),
            "산학협력연구동(E)": (10, 169),
        }


    # 위에서 정리한 정보를 바탕으로 새로운 데이터프레임에 각 열을 추가합니다
    for column_name, (row_start, col_idx) in columns_info.items():
        new_data[column_name] = original_data.iloc[row_start:row_start + 24, col_idx]

    # 파일 제목에서 날짜 형식의 값을 뽑아냅니다
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    date_match = re.search(date_pattern, file_path)
    if date_match:
        date_value = date_match.group(0)
    else:
        date_value = None

    # 'day' 열을 추가하고 날짜 값을 모든 행에 동일하게 적용합니다
    new_data["day"] = date_value

    # 'time' 열의 값에서 '시' 문자를 제거하고 숫자로 변환합니다
    new_data["time"] = new_data["time"].str.replace("시", "").astype(int)

    # day 열을 가장 왼쪽으로 이동합니다
    new_data = new_data[["day"] + [col for col in new_data.columns if col != "day"]]

    # 최종 결과를 CSV 형태로 저장합니다
    csv_output_path =  "./" + os.path.dirname(file_path) + "/processed/" + os.path.basename(file_path)[:-4] + '_filtered.csv'
    new_data.to_csv(csv_output_path, index=False)

In [ ]:
for file_name in file_master_list:
    if file_name.startswith('.'):
        continue
    preprocess(os.path.join(file_master_dir, file_name), 'master')

In [ ]:
for file_name in file_under_list:
    if file_name.startswith('.'):
        continue
    preprocess(os.path.join(file_under_dir, file_name), 'under')

In [ ]:
file_dir = './data-master/processed/'
file_list = os.listdir(file_dir)
file_list

master_dfs = [pd.read_csv(os.path.join(file_dir, file)) for file in file_list if not file.startswith('.')]
for df in master_dfs:
    for col in df.columns:
        if col not in ["day", "time"]:
            df.rename(columns={col: col + "_석사"}, inplace=True)
            
master_combined = pd.concat(master_dfs, ignore_index=True)


file_dir = './data-under/processed/'
file_list = os.listdir(file_dir)
file_list
under_dfs = [pd.read_csv(os.path.join(file_dir, file)) for file in file_list if not file.startswith('.')]
for df in under_dfs:
    for col in df.columns:
        if col not in ["day", "time"]:
            df.rename(columns={col: col + "_학사"}, inplace=True)

# Concatenate the "학사 일보*" files
under_combined = pd.concat(under_dfs, ignore_index=True)

# Merge the data based on 'day' and 'time'
merged_df = pd.merge(master_combined, under_combined, on=['day', 'time'], how='outer')
sorted_df = merged_df.sort_values(by=['day', 'time'])

In [ ]:

# Apply the function to the 'day' column to create a new 'weekday' column
sorted_df['weekday'] = sorted_df['day'].apply(get_day_of_week)

# Rearrange the columns to place 'weekday' next to 'day'
cols = sorted_df.columns.tolist()
cols = cols[:2] + [cols[-1]] + cols[2:-1]
sorted_df = sorted_df[cols]

In [ ]:
result_df = pd.read_csv("../solar-power/merged_result.csv")

relevant_cols = result_df[['day', 'time', '1시간기온', '1시간강수량', '일최고기온', '일최저기온']]

relevant_cols.rename(columns={
    '1시간기온': '1시간기온',
    '1시간강수량': '1시간강수량',
    '일최고기온': '일최고기온',
    '일최저기온': '일최저기온'
}, inplace=True)

merged_with_relevant = pd.merge(sorted_df, relevant_cols, on=['day', 'time'], how='left')

final_cols = merged_with_relevant.columns.tolist()
cols_order = final_cols[:3] + final_cols[-4:] + final_cols[3:-4]
final_df = merged_with_relevant[cols_order]

In [ ]:
# Save the merged dataframe to a CSV file
output_path = "./merged_data.csv"
final_df.to_csv(output_path, index=False)